In [3]:
import pandas as pd
import unicodedata
import os

# Always create the directory in case it's missing
os.makedirs('data/cleaned/sofifa', exist_ok=True)

# Load the Excel file
df = pd.read_excel('data/cleaned/sofifa/sofifa_players_cleaned.xlsx')

exclude_col = 'picture'

def remove_accents(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text)
        return ''.join(c for c in text if not unicodedata.combining(c))
    return text

def clean_string(value):
    if isinstance(value, str):
        value = remove_accents(value)
        value = value.strip().lower().replace(" ", "_")
    return value

def clean_dataframe(df, exclude_columns=None):
    if exclude_columns is None:
        exclude_columns = []

    # Clean column names
    df.columns = [clean_string(col) for col in df.columns]

    # Clean object columns except excluded
    for col in df.select_dtypes(include=['object']).columns:
        if col not in exclude_columns:
            df[col] = df[col].apply(clean_string)
    
    return df

# Remove dots in the 'name' column if it exists
if 'name' and 'team' in df.columns:
    df['name'] = df['name'].str.replace('.', '', regex=False)
    df['team'] = df['team'].str.replace('.', '', regex=False)

df_cleaned = clean_dataframe(df, exclude_columns=[exclude_col])
# Save cleaned version
output_path = 'data/cleaned/sofifa/sofifa_players_cleaned.xlsx'
df_cleaned.to_excel(output_path, index=False)
print(f"✅ Data cleaned and saved to: {output_path}")

✅ Data cleaned and saved to: data/cleaned/sofifa/sofifa_players_cleaned.xlsx


In [5]:
import pandas as pd
from fuzzywuzzy import fuzz

# Load the two Excel files
df1 = pd.read_excel("data/cleaned/sofascore/sofascore_preprocessed_teams_players.xlsx")  # contains column 'name'
df2 = pd.read_excel("data/cleaned/sofifa/sofifa_players_cleaned.xlsx")  # contains column 'name'

# Combine and normalize names
list1 = df1['name'].str.strip().str.lower().tolist()
list2 = df2['name'].str.strip().str.lower().tolist()
combined = list1 + list2

# Remove exact duplicates
unique_names = []
for name in combined:
    if name not in unique_names:
        unique_names.append(name)

# Fuzzy match and keep best version
final_names = []
while unique_names:
    name = unique_names.pop(0)
    duplicates = [n for n in unique_names if fuzz.ratio(name, n) >= 90]
    all_versions = [name] + duplicates
    best = max(all_versions, key=len)
    final_names.append(best)
    unique_names = [n for n in unique_names if n not in duplicates]

# Save to Excel
final_df = pd.DataFrame(final_names, columns=['unique_player_name'])
final_df.to_excel("merged_unique_names.xlsx", index=False)

print("✅ Merged and saved as merged_unique_names.xlsx")


KeyboardInterrupt: 